<a href="https://colab.research.google.com/github/Ashail33/Masters-work/blob/master/Clustering%20data%20creation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# !pip uninstall -y mdcgenpy
!pip install git+https://github.com/Ashail33/mdcgenpy.git

from  mdcgenpy import clusters as cl
import numpy as np
import pandas as pd
import json
import os
import glob
import re
from google.oauth2 import service_account
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from googleapiclient.http import MediaFileUpload
# import cl
from google.colab import drive



# The properties I will be adding to the dataset are:
#     1) Outliers - Binary ( two options) - outliers
#     2) Noise - Binary ( two options) add_noise=0,n_noise=None,
#     3) Number of clusters - I will select three values k
#     4) Number of data points - I will select three values ( 100 000, 1 000 000, 100 000 000)n_samples
#     5) Number of features - I will select five values ( 2, 10, 50, 100, 500) n_feats
#     6) Density - I will select three values compactness_factor
#     --7) Cluster shape - I will select three types - clusters within a radius ,  hollow shaped clusters , s-shaped / c-shaped clusters
#      distributions
#         'uniform': lambda shape, param: np.random.uniform(-param, param, shape),
#     'gaussian': lambda shape, param: np.random.normal(0, param, shape),
#     'logistic': lambda shape, param: np.random.logistic(0, param, shape),
#     'triangular': lambda shape, param: np.random.triangular(-param, 0, param, shape),
#     'gamma': lambda shape, param: np.random.gamma(2 + 8 * np.random.rand(), param / 5, shape),
#     'gap': lambda shape, param: gap(shape, param)

# --8) Missing values - this will need to be created by randomly removing values up to a certain number of columns and records


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/Ashail33/mdcgenpy.git to /tmp/pip-req-build-fuox9pnl
  Running command git clone --filter=blob:none --quiet https://github.com/Ashail33/mdcgenpy.git /tmp/pip-req-build-fuox9pnl
  Resolved https://github.com/Ashail33/mdcgenpy.git to commit b326c7f417b2f544b749ac5d59af56121cf32a9f
  Preparing metadata (setup.py) ... done
  Created wheel for mdcgenpy: filename=mdcgenpy-1.0.0-py3-none-any.whl size=24373 sha256=fb8f8c9de0aaef38e937d445115042724bc0e6fa3e16ec660f8a6b85f2df6877
  Stored in directory: /tmp/pip-ephem-wheel-cache-y36do4d6/wheels/95/95/3b/08d346ba5d5919db2dec23edd06e8ce312c71f6f93c6ebec06
Successfully built mdcgenpy


In [7]:
#keep this

drive.mount('/content/gdrive', force_remount=False)

# Set the path to the 'Masters_data' folder in your Google Drive
base_path = '/content/gdrive/MyDrive/'
folder_name = 'Masters_data'
folder_path = os.path.join(base_path, folder_name)

# Create the 'Masters_data' folder if it doesn't exist
if not os.path.exists(folder_path):
    os.makedirs(folder_path)

# Parameters for generating datasets
outliers = [0,500]
noise_options = [(0, None), (10, 100)]
n_clusters = [3, 5, 10]
n_samples = [10000,50000, 100000]
n_feats = [2, 10,100]
compactness_factors = [0.01, 0.5, 1]

# Distributions for cluster shapes
dist_options = ['gaussian']

# Find the last generated dataset
existing_files = glob.glob(os.path.join(folder_path, 'dataset_*.csv'))
dataset_numbers = [int(re.search(r'dataset_(\d+).csv', file).group(1)) for file in existing_files]
if dataset_numbers:
    last_dataset_id = max(dataset_numbers)
else:
    last_dataset_id = -1

metadata = []

# Continue from the last generated dataset
for idx, outlier in enumerate(outliers):
    for idy, (add_noise, n_noise) in enumerate(noise_options):
        for idz, k in enumerate(n_clusters):
            for idw, n_sample in enumerate(n_samples):
                for idv, n_feat in enumerate(n_feats):
                    for idu, compactness_factor in enumerate(compactness_factors):
                        for idt, distribution in enumerate(dist_options):
                            current_dataset_id = (idx * len(noise_options) * len(n_clusters) * len(n_samples) * len(n_feats) * len(compactness_factors) * len(dist_options) +
                                                  idy * len(n_clusters) * len(n_samples) * len(n_feats) * len(compactness_factors) * len(dist_options) +
                                                  idz * len(n_samples) * len(n_feats) * len(compactness_factors) * len(dist_options) +
                                                  idw * len(n_feats) * len(compactness_factors) * len(dist_options) +
                                                  idv * len(compactness_factors) * len(dist_options) +
                                                  idu * len(dist_options) +
                                                  idt)
                            
                            if current_dataset_id <= last_dataset_id:
                                continue

                            file_name = f'dataset_{current_dataset_id}.csv'
                            file_path = os.path.join(folder_path, file_name)
                            distributions = [distribution] * k
                        

                            cluster_gen = cl.ClusterGenerator(
                                          n_samples=n_sample,
                                          outliers=outlier,
                                          n_feats=n_feat,
                                          k=k,
                                          distributions=distributions,
                                          compactness_factor=compactness_factor,

                                          )

                            data = cluster_gen.generate_data(output_file=file_path)
                        

                            # distributions = [distribution] * k
                            # cluster_gen = cl.ClusterGenerator(
                            #     n_samples=n_sample,
                            #     outliers=outlier,
                            #     n_feats=n_feat,
                            #     k=k,
                            #     distributions=distributions,
                            #     compactness_factor=compactness_factor,
                            #     add_noise=add_noise,
                            #     n_noise=n_noise
                            # )
                            # data = cluster_gen.generate_data()

                            # data_combined = np.concatenate(data, axis=1)
                            # df = pd.DataFrame(data_combined)
                            
                            # file_name = f'dataset_{current_dataset_id}.csv'
                            # file_path = os.path.join(folder_path, file_name)
                            # df.to_csv(file_path, index=False)

                            dataset_properties = {
                                'id': current_dataset_id,
                                'outliers': outlier,
                                'add_noise': add_noise,
                                'n_noise': n_noise,'n_clusters': k,
                                'n_samples': n_sample,
                                'n_feats': n_feat,
                                'compactness_factor': compactness_factor,
                                'distribution': distribution,
                                'file_path': file_path
                                }

                            metadata.append(dataset_properties)

                        # del df
                        # del data_combined
                        # del data
                        # del cluster_gen
                            metadata_file_path = os.path.join(folder_path, 'metadata.json')
                            with open(metadata_file_path, 'w') as f:
                                json.dump(metadata, f)


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
called
3626
5222
1152
saved
called
3626
5222
1152
saved
called
3626
5222
1152
saved
called
3583
5274
1143
saved
called
3583
5274
1143
saved
called
3583
5274
1143
saved
called
3589
5281
1130
saved
called
3589
5281
1130
saved
called
3589
5281
1130
saved
called
18319
26095
5586
saved
called
18319
26095
5586
saved
called
18319
26095
5586
saved
called
18298
26096
5606
saved
called
18298
26096
5606
saved
called
18298
26096
5606
saved
called
18137
26283
5580
saved
called
18137
26283
5580
saved
called
18137
26283
5580
saved
called
36803
52097
11100
saved
called
36803
52097
11100
saved
called
36803
52097
11100
saved
called
36776
52121
11103
saved
called
36776
52121
11103
saved
called
36776
52121
11103
saved
called
36377
52464
11159
saved
called
36377
52464
11159
saved
called
36377
52464
11159
saved
called
2686
3825
696
1901
892
saved
called
2686
3825
696
1901
892
sa

In [7]:
# ################this code creates a way to get the metadata created if the file disappears 
# import json
# import os
# import glob
# import re

# # Set the path to the folder in your Google Drive
# base_path = '/content/gdrive/MyDrive/'
# folder_name = 'Masters_data'
# folder_path = os.path.join(base_path, folder_name)

# # Create the folder if it doesn't exist
# if not os.path.exists(folder_path):
#     os.makedirs(folder_path)

# # Find all the existing CSV files
# existing_files = glob.glob(os.path.join(folder_path, 'dataset_*.csv'))
# dataset_numbers = [int(re.search(r'dataset_(\d+).csv', file).group(1)) for file in existing_files]
# # Parameters for generating datasets
# outliers = [0, 500]
# noise_options = [(0, None), (0.1, 100)]
# n_clusters = [3, 5, 10]
# n_samples = [100_000, 1_000_000, 100_000_000]
# n_feats = [2, 10, 50, 100, 500]
# compactness_factors = [0.1, 0.5, 1]

# # Distributions for cluster shapes
# dist_options = ['uniform', 'gaussian', 'logistic', 'triangular', 'gamma', 'gap']
# metadata = []

# # Iterate through all combinations of parameters
# for idx, outlier in enumerate(outliers):
#     for idy, (add_noise, n_noise) in enumerate(noise_options):
#         for idz, k in enumerate(n_clusters):
#             for idw, n_sample in enumerate(n_samples):
#                 for idv, n_feat in enumerate(n_feats):
#                     for idu, compactness_factor in enumerate(compactness_factors):
#                         for idt, distribution in enumerate(dist_options):
#                             current_dataset_id = (idx * len(noise_options) * len(n_clusters) * len(n_samples) * len(n_feats) * len(compactness_factors) * len(dist_options) +
#                                                   idy * len(n_clusters) * len(n_samples) * len(n_feats) * len(compactness_factors) * len(dist_options) +
#                                                   idz * len(n_samples) * len(n_feats) * len(compactness_factors) * len(dist_options) +
#                                                   idw * len(n_feats) * len(compactness_factors) * len(dist_options) +
#                                                   idv * len(compactness_factors) * len(dist_options) +
#                                                   idu * len(dist_options) +
#                                                   idt)
                            
#                             if current_dataset_id in dataset_numbers:
#                                 file_path = os.path.join(folder_path, f'dataset_{current_dataset_id}.csv')
#                                 dataset_properties = {
#                                     'id': current_dataset_id,
#                                     'outliers': outlier,
#                                     'add_noise': add_noise,
#                                     'n_noise': n_noise,
#                                     'n_clusters': k,
#                                     'n_samples': n_sample,
#                                     'n_feats': n_feat,
#                                     'compactness_factor': compactness_factor,
#                                     'distribution': distribution,
#                                     'file_path': file_path
#                                 }

#                                 metadata.append(dataset_properties)

# # Save metadata to a JSON file
# metadata_file_path = os.path.join(folder_path, 'metadata.json')
# with open(metadata_file_path, 'w') as f:
#     json.dump(metadata, f)


# import pandas as pd

# # Load metadata from the JSON file
# metadata_file_path = os.path.join(folder_path, 'metadata.json')
# with open(metadata_file_path, 'r') as f:
#     metadata = json.load(f)

# # Convert metadata to a DataFrame
# metadata_df = pd.DataFrame(metadata)

# # Save metadata DataFrame to a CSV file
# metadata_csv_path = os.path.join(folder_path, 'metadata.csv')
# metadata_df.to_csv(metadata_csv_path, index=False)


In [6]:
###################This deletes all files in the folder
# import os
# import glob
# drive.mount('/content/gdrive', force_remount=False)
# # Set the path to the 'Masters_data' folder in your Google Drive
# base_path = '/content/gdrive/MyDrive/'
# folder_name = 'Masters_data'
# folder_path = os.path.join(base_path, folder_name)

# # Find all the existing dataset CSV files and delete them
# existing_files = glob.glob(os.path.join(folder_path, 'dataset_*.csv'))
# print(existing_files)
# for file in existing_files:
#     print(file)
#     os.remove(file)

# # Delete the metadata JSON and CSV files
# metadata_json_path = os.path.join(folder_path, 'metadata.json')
# metadata_csv_path = os.path.join(folder_path, 'metadata.csv')

# if os.path.exists(metadata_json_path):
#     os.remove(metadata_json_path)

# if os.path.exists(metadata_csv_path):
#     os.remove(metadata_csv_path)


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
[]
